In [0]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction

This is the second part of a two-part group of Python Notebooks based on an Earth Engine tutorial. (Links below.) 

It downloads data from Google Cloud (generated by the previous notebook) then uses it to train a model, then uses that model to make predictions using LANDSAT 7 data.

# Setup software libraries

Authenticate and import as necessary.

In [0]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [13]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=eiJOIaOdSYvrsxXI4c2yl54_oXRtOs2yV7o8HUsDjRk&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/zAFwwfT2PnBoN1x4OohzmETrccy2zJnE-zqLKdB8qxhWXPrF7Pibw1A

Successfully saved authorization token.


In [16]:
# Tensorflow setup.
%tensorflow_version 1.x
import tensorflow as tf

# tf.enable_eager_execution() ## enabled by default
print(tf.__version__)

1.15.2


In [17]:
# Folium setup.
import folium
print(folium.__version__)

0.8.3


# Variables

Declare the variables that will be in use throughout the notebook.

## Specify your Cloud Storage Bucket
You must have write access to a bucket to run this demo!  To run it read-only, use the demo bucket below, but note that writes to this bucket will not work.

In [0]:
# INSERT YOUR BUCKET HERE:
BUCKET = 'mangroves_model'

## Set other global variables

In [0]:
# Specify names locations for outputs in Cloud Storage. 
FOLDER = 'fcnn-caribbean-mangroves/data-model-3'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'
PRED_FOLDER = FOLDER + '/predictions'

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
thermalBands = [] ## Not on Lansat 7
BANDS = opticalBands + thermalBands
RESPONSE = 'constant'
FEATURES = BANDS + [RESPONSE]
RESOLUTION = 30

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Specify model training parameters.
BATCH_SIZE = 32
EPOCHS = 100
BUFFER_SIZE = 2000
OPTIMIZER = 'SGD'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']

YEAR = "2007"

TEST_LOC = [8.527453749657104,-83.29948528935525]
CARIBBEAN_GEO = ee.Geometry.Polygon(
         [[-117, 32],
          [-117, 0],
          [-53, 0],
          [-53, 32]]);

COSTA_RICA = ee.Geometry.Polygon(
        [[[-85.76142594735113, 10.788598674810393],
          [-85.72846696297613, 10.64826826510741],
          [-85.86030290047613, 10.605076632983748],
          [-85.87128922860113, 10.01058363425571],
          [-85.14619157235113, 9.480023700661231],
          [-84.93745133797613, 9.685850985223768],
          [-84.73969743172613, 9.599202089022581],
          [-83.67402360360113, 8.970346670767807],
          [-83.81684586922613, 8.525148131877797],
          [-82.88300797860113, 8.046797290019212],
          [-83.02583024422613, 8.329529569369285],
          [-82.80610368172613, 8.459952952528822],
          [-82.93793961922613, 8.785816842891018],
          [-82.70722672860113, 8.937789343784477],
          [-82.93793961922613, 9.100546675686568],
          [-82.93793961922613, 9.436675811729556],
          [-82.85004899422613, 9.620866398800096],
          [-82.66328141610113, 9.480023700661231],
          [-82.56440446297613, 9.577536393773666],
          [-83.67692015217513, 10.95060893139355],
          [-83.66868040608138, 10.810354208798493],
          [-83.87192747639388, 10.718613684754791],
          [-84.02024290608138, 10.772582081856763],
          [-84.10264036701888, 10.764487437639806],
          [-84.36081907795638, 10.993750909979108],
          [-84.68216917561263, 11.085406634790713],
          [-84.91837523030013, 10.945215741587166],
          [-85.60502073811263, 11.222836305098825],
          [-86.03781948005137, 10.877434114738056]]]);

# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.  The following are helper functions for that.

In [0]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns: 
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns: 
    A dtuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns: 
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

Use the helpers to read in the training dataset.  Print the first record to check.

In [0]:
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset

training = get_training_dataset()

print(iter(training.take(1)).next())

(<tf.Tensor: shape=(16, 256, 256, 7), dtype=float32, numpy=
array([[[[0.0344 , 0.049  , 0.0466 , ..., 0.1085 , 0.29625, 0.05785],
         [0.03545, 0.0474 , 0.04505, ..., 0.1155 , 0.29625, 0.0603 ],
         [0.0354 , 0.04475, 0.04445, ..., 0.1154 , 0.2965 , 0.0614 ],
         ...,
         [0.0325 , 0.0548 , 0.03915, ..., 0.11655, 0.297  , 0.0491 ],
         [0.03025, 0.05435, 0.04005, ..., 0.12075, 0.297  , 0.05415],
         [0.0312 , 0.0522 , 0.0388 , ..., 0.1188 , 0.297  , 0.0494 ]],

        [[0.03255, 0.04715, 0.0471 , ..., 0.11955, 0.29625, 0.061  ],
         [0.0339 , 0.0487 , 0.0478 , ..., 0.1299 , 0.296  , 0.0685 ],
         [0.0346 , 0.0476 , 0.0479 , ..., 0.1226 , 0.296  , 0.0656 ],
         ...,
         [0.0292 , 0.0536 , 0.04   , ..., 0.1154 , 0.297  , 0.0464 ],
         [0.0292 , 0.0516 , 0.0366 , ..., 0.10985, 0.297  , 0.0452 ],
         [0.0312 , 0.0546 , 0.0439 , ..., 0.1184 , 0.297  , 0.0505 ]],

        [[0.0338 , 0.0487 , 0.0475 , ..., 0.117  , 0.296  , 0.061  ]

# Evaluation data

Now do the same thing to get an evaluation dataset.  Note that unlike the training dataset, the evaluation dataset has a batch size of 1, is not repeated and is not shuffled.

In [0]:
def get_eval_dataset():
	"""Get the preprocessed evaluation dataset
  Returns: 
    A tf.data.Dataset of evaluation data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + EVAL_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

evaluation = get_eval_dataset()

# Model

Here we use the Keras implementation of the U-Net model.  The U-Net model takes 256x256 pixel patches as input and outputs per-pixel class probability, label or a continuous output.  We can implement the model essentially unmodified, but will use mean squared error loss on the sigmoidal output since we are treating this as a regression problem, rather than a classification problem.  Since impervious surface fraction is constrained to [0,1], with many values close to zero or one, a saturating activation function is suitable here.

In [0]:
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers

def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER), 
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

# Training the model

You train a Keras model by calling `.fit()` on it.  Here we're going to train for 10 epochs, which is suitable for demonstration purposes.  For production use, you probably want to optimize this parameter, for example through [hyperparamter tuning](https://cloud.google.com/ml-engine/docs/tensorflow/using-hyperparameter-tuning).

In [0]:
m = get_model()

m.fit(
    x=training, 
    epochs=EPOCHS, 
    steps_per_epoch=int(trainCount / BATCH_SIZE), 
    validation_data=evaluation,
    validation_steps=evalCount)

Epoch 1/10
81/81 [==============================] - 57s 698ms/step - loss: 0.1382 - root_mean_squared_error: 0.3718 - val_loss: 0.1720 - val_root_mean_squared_error: 0.4147
Epoch 2/10
81/81 [==============================] - 58s 721ms/step - loss: 0.0828 - root_mean_squared_error: 0.2878 - val_loss: 0.1256 - val_root_mean_squared_error: 0.3545
Epoch 3/10
81/81 [==============================] - 57s 703ms/step - loss: 0.0611 - root_mean_squared_error: 0.2472 - val_loss: 0.1063 - val_root_mean_squared_error: 0.3260
Epoch 4/10
81/81 [==============================] - 56s 685ms/step - loss: 0.0521 - root_mean_squared_error: 0.2283 - val_loss: 0.0959 - val_root_mean_squared_error: 0.3096
Epoch 5/10
81/81 [==============================] - 30s 369ms/step - loss: 0.0635 - root_mean_squared_error: 0.2520 - val_loss: 0.0808 - val_root_mean_squared_error: 0.2842
Epoch 6/10
81/81 [==============================] - 93s 1s/step - loss: 0.0597 - root_mean_squared_error: 0.2423 - val_loss: 0.0655 - v

Now you can either use that model for predictions, or run the following cell in order to load another model from Google Cloud

In [20]:
# Load a model trained in Google Cloud, with RMSE ~0.15
MODEL_DIR = 'gs://mangroves_model/fcnn-model-3/trainer_balanced/model'
m = tf.contrib.saved_model.load_keras_model(MODEL_DIR)
m.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 2048        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Prediction

The prediction pipeline is:

1.  Export imagery on which to do predictions from Earth Engine in TFRecord format to a Cloud Storge bucket.
2.  Use the trained model to make the predictions.
3.  Write the predictions to a TFRecord file in a Cloud Storage.
4.  Upload the predictions TFRecord file to Earth Engine.

The following functions handle this process.  It's useful to separate the export from the predictions so that you can experiment with different models without running the export every time.

In [0]:
def getImage(year):
  # Use Landsat 8 surface reflectance data.
  l8sr = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')

  # Cloud masking function.
  def maskL8sr(image):
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()
    qa = image.select('pixel_qa')
    mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
      qa.bitwiseAnd(cloudsBitMask).eq(0))
    mask2 = image.mask().reduce('min')
    mask3 = image.select(opticalBands).gt(0).And(
            image.select(opticalBands).lt(10000)).reduce('min')
    mask = mask1.And(mask2).And(mask3)
    return image.select(opticalBands).divide(10000).updateMask(mask)

  # The image input data is a cloud-masked median composite.
  print("Getting image for is "+str(year))
  return l8sr.filterDate(year+'-01-01', year+'-12-31').map(maskL8sr).median()


def doExport(out_image_base, kernel_buffer, region, year):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toCloudStorage(
    image = getImage(year).select(BANDS), 
    description = out_image_base + '_' + year, 
    bucket = BUCKET, 
    fileNamePrefix = PRED_FOLDER + '/' + out_image_base + '_' + year, 
    region = region.getInfo()['coordinates'], 
    scale = RESOLUTION, 
    fileFormat = 'TFRecord', 
    maxPixels = 1e10,
    formatOptions = { 
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(15)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [0]:
def doPrediction(out_image_base, user_folder, kernel_buffer, region, year):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print(f"Looking for TFRecord files in gs://{BUCKET}/{PRED_FOLDER}")
  
  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{BUCKET}'/'{PRED_FOLDER}
  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if (out_image_base + '_' + year) in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(imageFilesList)
  print(jsonFile)
  
  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']
  
  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(d):
    inputsList = [d.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked
  
   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)
  
  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  out_image_file = 'gs://' + BUCKET + '/' + PRED_FOLDER + '/' + out_image_base + '_' + year + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'impervious': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base + '_' + year
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

Now there's all the code needed to run the prediction pipeline, all that remains is to specify the output region in which to do the prediction, the names of the output files, where to put them, and the shape of the outputs.  In terms of the shape, the model is trained on 256x256 patches, but can work (in theory) on any patch that's big enough with even dimensions ([reference](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf)).  Because of tile boundary artifacts, give the model slightly larger patches for prediction, then clip out the middle 256x256 patch.  This is controlled with a kernel buffer, half the size of which will extend beyond the kernel buffer.  For example, specifying a 128x128 kernel will append 64 pixels on each side of the patch, to ensure that the pixels in the output are taken from inputs completely covered by the kernel.  

In [0]:
# Output assets folder: YOUR FOLDER
user_folder = 'users/pandringa' # INSERT YOUR FOLDER HERE.

# Base file name to use for TFRecord files and assets.
bj_image_base = 'costa_rica'
# Half this will extend on the sides of each patch.
bj_kernel_buffer = [128, 128]

In [0]:
# Run the export.
doExport(bj_image_base, bj_kernel_buffer, COSTA_RICA, "2007")

Getting image for is 2007
Running image export to Cloud Storage...


In [0]:
# Run the prediction.
doPrediction(bj_image_base, user_folder, bj_kernel_buffer, COSTA_RICA, "2007")

# Display the output

One the data has been exported, the model has made predictions and the predictions have been written to a file, and the image imported to Earth Engine, it's possible to display the resultant Earth Engine asset.  Here, display the impervious area predictions over Beijing, China.

In [0]:
out_image = ee.Image(user_folder + '/' + bj_image_base)
mapid = out_image.getMapId({'min': 0, 'max': 1})
map = folium.Map(location=TEST_LOC)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='predicted impervious',
  ).add_to(map)
map.add_child(folium.LayerControl())
map